In [1]:
import os

from dotenv import load_dotenv
load_dotenv()

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq

In [7]:
import sqlite3
import ollama
from typing import TypedDict

from langgraph.graph import StateGraph,END
from langchain_community.tools import DuckDuckGoSearchRun

#### Loacl DB
# 1. Set up SQLite database and store sample documents
def setup_database():
    conn = sqlite3.connect('myfile.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS documents (
                        id INTEGER PRIMARY KEY,
                        content TEXT
                      )''')
    sample_documents = [
        ("The first letter is alpha.",),
        ("The second letter is beta.",)
    ]
    cursor.executemany("INSERT INTO documents (content) VALUES (?)", sample_documents)
    conn.commit()
    conn.close()

setup_database()

# 2. Function to retrieve documents based on query
def retrieve_documents(query):
    conn = sqlite3.connect('myfile.db')
    cursor = conn.cursor()
    cursor.execute("SELECT content FROM documents WHERE content LIKE ?", ('%' + query + '%',))
    results = cursor.fetchall()
    conn.close()
    return [result[0] for result in results]

#####
#  Search tool - DuckDuckGo
search_tool = DuckDuckGoSearchRun()

###
# Agent State
class AgentState(TypedDict):
    question: str
    local_docs: str
    search_result: str
    final_answer: str


## Node-1 - Check local database 

def local_db_node(state: AgentState):
    question = state['question']
    docs = retrieve_documents(question)
    if docs:
        context = "\n".join(docs)
        prompt = f'''
        Answer the question using only the context below.
        Context:
        {context}
        Question:
        {question}
        '''
        response = ollama.chat(model="gemma2:2b", messages=[{"role": "user", "content": prompt}])
        return {
            "local_docs": context,
            "final_answer": response["message"]["content"]
        }
    return { "local_docs": "","final_answer": "" }

def route_after_local(state: AgentState):
    if state["final_answer"]:
        return END
    return "search_node"

## Node - external search 
def search_node(state: AgentState):
    question = state["question"]
    search_result = search_tool.run(question)
    return {
        "search_result": search_result
    }

## Node - Final Answer using search result
def final_answer_node(state: AgentState):
    prompt = f"""The answer NOT found in the local database.
    use external search result to ansewr clearly.
    search result:
    {state["search_result"]}
    Question:
    {state["question"]}
    """
    response = ollama.chat(model="gemma2:2b", messages=[{"role": "user", "content": prompt}])
    return { "final_answer": response["message"]["content"] }
    

## Build the Graph
graph = StateGraph(AgentState)
graph.add_node("local_node",local_db_node)
graph.add_node("search_node",search_node)
graph.add_node("final_node",final_answer_node)
graph.set_entry_point("local_node")
graph.add_conditional_edges(
    "local_node",
    route_after_local
)
graph.add_edge("search_node","final_node")
graph.add_edge("final_node",END)
app = graph.compile()

## Run Agent

def ask_agent(question: str):
    result = app.invoke({
        "question": question,
        "local_docs": "",
        "search_result": "",
        "final_answer": ""
    })
    return result ["final_answer"]

In [8]:
## Test
print("Local DB Question:")
print(ask_agent("first letter"))

print("\nExternal Search Question:")
print(ask_agent("Who invented Python Programming language?"))

Local DB Question:
alpha 


External Search Question:
Python programming language was invented by **Guido van Rossum**. 



In [18]:
import sqlite3
from typing import TypedDict

from groq import Groq
from langgraph.graph import StateGraph, END
from langchain_community.tools import DuckDuckGoSearchRun


# Groq client

client = Groq(api_key=os.getenv('GROQ_API_KEY'))

MODEL_NAME = "llama-3.1-8b-instant"   


# Local DB
# ---------------------------------
def setup_database():
    conn = sqlite3.connect("myfile.db")
    cursor = conn.cursor()
    cursor.execute(
        """CREATE TABLE IF NOT EXISTS documents (
            id INTEGER PRIMARY KEY,
            content TEXT
        )"""
    )
    sample_documents = [
        ("The first letter is alpha.",),
        ("The second letter is beta.",)
    ]
    cursor.executemany(
        "INSERT INTO documents (content) VALUES (?)",
        sample_documents
    )
    conn.commit()
    conn.close()

setup_database()

def retrieve_documents(query):
    conn = sqlite3.connect("myfile.db")
    cursor = conn.cursor()
    cursor.execute(
        "SELECT content FROM documents WHERE content LIKE ?",
        ("%" + query + "%",)
    )
    results = cursor.fetchall()
    conn.close()
    return [r[0] for r in results]


# Search Tool
# ---------------------------------
search_tool = DuckDuckGoSearchRun()

# ---------------------------------
# Agent State
# ---------------------------------
class AgentState(TypedDict):
    question: str
    local_docs: str
    search_result: str
    final_answer: str


# Node 1: Local DB 
# ---------------------------------
def local_db_node(state: AgentState):
    question = state["question"]
    docs = retrieve_documents(question)

    if docs:
        context = "\n".join(docs)
        prompt = f"""
Answer the question using only the context below.

Context:
{context}

Question:
{question}
"""
        response = client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        #print(response)
        return {
            "local_docs": context,
            "final_answer": response.choices[0].message.content
        }

    return {
        "local_docs": "",
        "final_answer": ""
    }

def route_after_local(state: AgentState):
    if state["final_answer"]:
        return END
    return "search_node"


# Node 2: External Search
# ---------------------------------
def search_node(state: AgentState):
    question = state["question"]
    search_result = search_tool.run(question)
    return {
        "search_result": search_result
    }


# Node 3: Final Answer using search
# ---------------------------------
def final_answer_node(state: AgentState):
    prompt = f"""
The answer was NOT found in the local database.
Use the external search result to answer clearly.

Search result:
{state["search_result"]}

Question:
{state["question"]}
"""

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
   
    return {
        "final_answer": response.choices[0].message.content
    }


# Build Graph
# ---------------------------------
graph = StateGraph(AgentState)

graph.add_node("local_node", local_db_node)
graph.add_node("search_node", search_node)
graph.add_node("final_node", final_answer_node)

graph.set_entry_point("local_node")

graph.add_conditional_edges(
    "local_node",
    route_after_local
)

graph.add_edge("search_node", "final_node")
graph.add_edge("final_node", END)

app = graph.compile()

# ---------------------------------
# Run Agent
# ---------------------------------
def ask_agent(question: str):
    result = app.invoke({
        "question": question,
        "local_docs": "",
        "search_result": "",
        "final_answer": ""
    })
    return result["final_answer"]


In [19]:
print("Local DB Question:")
print(ask_agent("first letter"))

Local DB Question:
alpha


In [20]:
print("\nExternal Search Question:")
print(ask_agent("Who invented Python Programming language?"))


External Search Question:
Based on the search result:

Python was invented by **Guido van Rossum** in the late 1980s and first released in 1991. He created the language to make programming simpler and more accessible.


In [39]:
import sqlite3
import ollama
from typing import TypedDict

from langgraph.graph import StateGraph,END
from langchain_community.tools import DuckDuckGoSearchRun

#### Database ( Docs + LongTerm Memory)
# 1. Set up SQLite database and store sample documents
def setup_database():
    conn = sqlite3.connect('agent_memory.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS documents (
                        id INTEGER PRIMARY KEY,
                        content TEXT
                      )''')
    # Long Term memory
    cursor.execute('''CREATE TABLE IF NOT EXISTS memory (
                        id INTEGER PRIMARY KEY,
                        question TEXT,
                        answer TEXT,
                        source TEXT
                      )''')
    docs = [
        ("The first letter is alpha.",),
        ("The second letter is beta.",)
    ]
    cursor.executemany("INSERT INTO documents (content) VALUES (?)", docs)
    conn.commit()
    conn.close()

setup_database()

# Long-Term Memory Functions

def recall_memory(question: str):
    conn = sqlite3.connect("agent_memory.db")
    cursor = conn.cursor()
    cursor.execute("select answer from memory where question like ?",(f"%{question}%",))
    result = cursor.fetchone(),
    conn.close()
    return result[0] if result else None

def store_memory(question: str,answer: str,source: str):
    conn = sqlite3.connect("agent_memory.db")
    cursor = conn.cursor()
    cursor.execute("insert into memory (question,answer,source) values(?,?,?)",(question,answer,source))
    conn.commit()
    conn.close()
    

# Local Knowledge retrieval

def retrieval_documents(query):
    conn = sqlite3.connect("agent_memory.db")
    cursor = conn.cursor()
    cursor.execute("select content from documents where content like ?",(f"%{query}%",))
    results = cursor.fetchall()
    conn.close()
    return [r[0] for r in results]

# 
#####
#  Search tool - DuckDuckGo
search_tool = DuckDuckGoSearchRun()

###
# Agent State
class AgentState(TypedDict):
    question: str
    local_docs: str
    search_result: str
    final_answer: str
    memory_answer: str

# Node-1 - Long-Term Memory Recall
def memory_node(state: AgentState):
    answer = recall_memory(state["question"])
    if answer:
        return {"final_answer":answer,"memory_answer":answer}
    return {"memory_answer": ""}


#  Node 2: Local Knowledge
def local_node(state: AgentState):
    docs = retrieve_documents(state["question"])
    if not docs:
        return {"local_answer": ""}

    context = "\n".join(docs)
    prompt = f"""
Answer ONLY using this context.

Context:
{context}

Question:
{state["question"]}
"""

    response = ollama.chat(
        model="gemma2:2b",
        messages=[{"role": "user", "content": prompt}]
    )

    return {
        "local_answer": response["message"]["content"],
        "final_answer": response["message"]["content"]
    }


#  Routing Logic
def route_after_memory(state: AgentState):
    if state["final_answer"]:
        return END
    return "local_node"


def route_after_local(state: AgentState):
    if state["final_answer"]:
        return END
    return "search_node"


#  Node 3: External Search
def search_node(state: AgentState):
    result = search_tool.run(state["question"])
    return {"search_result": result}

#  Node 4: Final Answer + Store Memory
def final_node(state: AgentState):
    prompt = f"""
Answer the question using external search results.

Search Results:
{state["search_result"]}

Question:
{state["question"]}
"""
    response = ollama.chat(
        model="gemma2:2b",
        messages=[{"role": "user", "content": prompt}]
    )
    answer = response["message"]["content"]

    # Store in long-term memory
    store_memory(
        state["question"],
        answer,
        source="duckduckgo"
    )
    return {"final_answer": answer}
    


In [40]:
# Build LangGraph
graph = StateGraph(AgentState)

graph.add_node("memory_node", memory_node)
graph.add_node("local_node", local_node)
graph.add_node("search_node", search_node)
graph.add_node("final_node", final_node)

graph.set_entry_point("memory_node")

graph.add_conditional_edges("memory_node", route_after_memory)
graph.add_conditional_edges("local_node", route_after_local)

graph.add_edge("search_node", "final_node")
graph.add_edge("final_node", END)

app = graph.compile()


In [41]:
# Run Agent
def ask_agent(question: str):
    result = app.invoke({
        "question": question,
        "memory_answer": "",
        "local_answer": "",
        "search_result": "",
        "final_answer": ""
    })
    print(result)
    return result["final_answer"]


In [42]:
answer = ask_agent("first")
print(answer)

{'question': 'first', 'search_result': '', 'final_answer': 'alpha \n', 'memory_answer': ''}
alpha 



In [43]:
answer = ask_agent("Who invented Python?")
print(answer)

{'question': 'Who invented Python?', 'search_result': '', 'final_answer': ('Guido van Rossum is the inventor of the Python programming language.',), 'memory_answer': ('Guido van Rossum is the inventor of the Python programming language.',)}
('Guido van Rossum is the inventor of the Python programming language.',)


In [44]:
answer = ask_agent("How to calculate sum of two digits?")
print(answer)

{'question': 'How to calculate sum of two digits?', 'search_result': '', 'final_answer': ('Here\'s how you can calculate the sum of two-digit numbers:\n\n**Understanding the Formula**\n\nThe key is to understand that a number with two digits (like 12)  can be seen as a single unit. We use an arithmetic series formula, like this: \n\n* **Sum = (n/2)(first digit + last digit)** \n\nwhere "n" represents the number of terms.\n\n\n**Let\'s break down the process using examples:**\n\n* **For example, let\'s calculate the sum of digits for the number 12:**\n    1. **Identify the first and last digits:**  First digit = 1, Last digit = 2\n    2. **Apply the formula:** Sum = (n/2)(1 + 2) \n       * We need to remember that n in this case is 2 (number of digits in the number 12). So:  Sum = (2/2)(1 + 2)  = 1 * 3\n    3. **Result:** Sum = 3\n\n**Important Notes:**\n\n\n* This formula works for any two-digit number, like 15, 47, 98, etc.\n* Make sure to use the correct units (like 10 for numbers).\

In [45]:
answer = ask_agent("first")
print(answer)

{'question': 'first', 'search_result': '', 'final_answer': 'alpha \n', 'memory_answer': ''}
alpha 



In [ ]:
### ReAct + Memory 

In [73]:
import sqlite3
import ollama
from typing import TypedDict

from langgraph.graph import StateGraph,END
from langchain_community.tools import DuckDuckGoSearchRun

#### Database ( Docs + LongTerm Memory)
# 1. Set up SQLite database and store sample documents
def setup_database():
    conn = sqlite3.connect('ReAct_memory.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS documents (
                        id INTEGER PRIMARY KEY,
                        content TEXT
                      )''')
    # Long Term memory
    cursor.execute('''CREATE TABLE IF NOT EXISTS memory (
                        id INTEGER PRIMARY KEY,
                        question TEXT,
                        answer TEXT
                      )''')
    docs = [
        ("The first letter is alpha.",),
        ("The second letter is beta.",)
    ]
    cursor.execute("delete from documents")
    cursor.executemany("INSERT INTO documents (content) VALUES (?)", docs)
    conn.commit()
    conn.close()

setup_database()

In [74]:
# memory components
# --------------------
def recall_memory(question: str):
    conn = sqlite3.connect("ReAct_memory.db")
    cursor = conn.cursor()
    cursor.execute("select answer from memory where question like ?",(f"%{question}%",))
    result = cursor.fetchone(),
    conn.close()
    return result[0] if result else None

def store_memory(question,answer):
    conn = sqlite3.connect("ReAct_memory.db")
    cursor = conn.cursor()
    cursor.execute("insert into memory (question,answer) values(?,?)",(question,answer))
    conn.commit()
    conn.close()

In [75]:
# Local Knowledge Retrieval
def retrieval_documents(query):
    conn = sqlite3.connect("ReAct_memory.db")
    cursor = conn.cursor()
    cursor.execute("select content from documents where content like ?",(f"%{query}%",))
    results = cursor.fetchall()
    conn.close()
    return [r[0] for r in results]


In [76]:
# #####
#  Search tool - DuckDuckGo
search_tool = DuckDuckGoSearchRun()

In [77]:
class AgentState(TypedDict):
    question: str
    thought: str
    observation: str
    final_answer: str

In [78]:
# Node-1 : THINK: MemoryCheck
def memory_THINK_node(state: AgentState):
    answer = recall_memory(state["question"])
    if answer:
        return { "thought":"I Found the answer in LTM","final_answer":answer}
    return {
        "thought": "No memory found. I should check local knowledge"
    }

In [79]:
#  Node 2: ACT: Local Knowledge
def local_ACT_node(state: AgentState):
    docs = retrieve_documents(state["question"])
    if not docs:
        return {"thought": "Local Knowledge insufficent. I Should Search the web."}
    context = "\n".join(docs)
    prompt = f"""
Answer ONLY using this context.

Context:
{context}

Question:
{state["question"]}
"""
    response = ollama.chat(
        model="gemma2:2b",
        messages=[{"role": "user", "content": prompt}]
    )
    answer = response["message"]["content"]
    store_memory(state['question'],answer)
    
    return {
       "thought": "Answered using local knowledge",
        "final_answer": answer
    }



In [80]:
# Node 3: - ACT : WebSearch
def search_ACT_node(state: AgentState):
    result = search_tool.run(state["question"])
    return {"observation": result}

In [81]:
# Node 4: FINAL Answer
def final_ANSWER_node(state: AgentState):
    prompt = f"""
Answer the question using the search observation below.
Observation:
{state["observation"]}

Question:
{state["question"]}
"""
    response = ollama.chat(
        model="gemma2:2b",
        messages=[{"role": "user", "content": prompt}]
    )
    answer = response["message"]["content"]
    store_memory(state["question"],answer)
    return {"thought": "Answered using web search.", "final_answer": answer}

In [82]:
# Routing Logic 
def route_after_memory(state: AgentState):
    if state.get('final_answer'):
        return END
    return "local_act"
        

def route_after_local(state: AgentState):
    if state.get('final_answer'):
        return END
    return "search_act"

In [83]:
# Build LangGraph
graph = StateGraph(AgentState)

graph.add_node("memory_think", memory_THINK_node)
graph.add_node("local_act", local_ACT_node)
graph.add_node("search_act", search_ACT_node)
graph.add_node("final_answer", final_ANSWER_node)

graph.set_entry_point("memory_think")

graph.add_conditional_edges("memory_think", route_after_memory)
graph.add_conditional_edges("local_act", route_after_local)

graph.add_edge("search_act", "final_answer")
graph.add_edge("final_answer", END)

app = graph.compile()

In [84]:
# Run the Agent
def ask(question):
    result = app.invoke({
        "question":question,
        "thought":"",
        "observation":"",
        "final_answer":""
    })
    print("\nTHOUGHT:",result.get("thought"))
    print("\nANSWER:",result.get("final_answer"))
    return result["final_answer"]

In [85]:
## Test - Query
ask("first letter")


THOUGHT: Answered using local knowledge

ANSWER: a 



'a \n'

In [86]:
ask("who invented Python programming language?")


THOUGHT: Answered using web search.

ANSWER: Python was invented by **Guido van Rossum**. 



'Python was invented by **Guido van Rossum**. \n'

In [87]:
ask("who invented Python programming language?") # memory hit


THOUGHT: I Found the answer in LTM

ANSWER: ('Python was invented by **Guido van Rossum**. \n',)


('Python was invented by **Guido van Rossum**. \n',)